# HTTP GET

## Basic HTTP GET using the Requests module. 

Note: Requests is not in the standard library, so you will need to install it unless your distribution already includes it (e.g. Anaconda).

In [ ]:
import requests

uri = 'http://bioimages.vanderbilt.edu/pages/contributors.htm'
#uri = 'http://api.open-notify.org/iss-now.json'
response = requests.get(uri)
print('Dereferenced URL: ', response.url)
print('HTTP status code: ', response.status_code)
# The .headers attribute returns a dictionary with keys that correspond to the header keys
print('Response media type: ', response.headers['content-type'])
print('Response body: ')
print()
print(response.text)

Try again with the URI http://api.open-notify.org/iss-now.json

## HTTP GET with content negotiation.  

If there is a redirect, requests generates a `.history` attribute that is a list of responses.

In [ ]:
import requests

uri = 'http://dbpedia.org/resource/Bonobo'
headerDict = {'Accept' : 'text/html'}
#headerDict = {'Accept' : 'application/ld+json'}
response = requests.get(uri, headers=headerDict)
if response.history:
    for redirect in response.history:
        print('Redirect type: ', redirect.status_code, ' from ', redirect.url)
print('Dereferenced URL: ', response.url)
print('HTTP status code: ', response.status_code)
print('Response media type: ', response.headers['content-type'])
print('Response body: ')
print()
print(response.text)

Try again with an `Accept` request header of `application/ld+json`

## Data from GitHub

Remember to use the URL of the raw file, not the data rendered as HTML.  The rendered page in this example is https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/data/president_heights.csv

This example provides a function `urlToListOfDicts()` that will turn an online CSV file into a list of dictionaries.

This script finds the shortest and tallest U.S. presidents (Donald Trump missing from dataset).

In [ ]:
import requests
import csv

def urlToListOfDicts(url):
    r = requests.get(url)
    fileText = r.text.split('\n')
    fileRows = csv.DictReader(fileText)
    dictData = []
    for row in fileRows:
        dictData.append(row)
    return dictData
    
uri = 'https://raw.githubusercontent.com/jakevdp/PythonDataScienceHandbook/master/notebooks/data/president_heights.csv'
data = urlToListOfDicts(uri)

tallHeight = 0
shortHeight = 300
for president in data:
    if int(president['height(cm)']) > tallHeight:
        tallGuy = president['name']
        tallHeight = int(president['height(cm)'])
    if int(president['height(cm)']) < shortHeight:
        shortGuy = president['name']
        shortHeight = int(president['height(cm)'])

print(tallGuy, 'is tallest at', tallHeight, 'cm')
print(shortGuy, 'is shortest at', shortHeight, 'cm')


# APIs

## Where's the ISS? API

The webbrowser module provides access to the default system browser.  The `.open_new_tab` method opens a new tab (launching the browser if it isn't already open).

In [ ]:
import requests
import webbrowser

url = 'http://api.open-notify.org/iss-now.json'
response = requests.get(url)
print('JSON text: ', response.text)
print()
data = response.json()
print('Python data structure: ', data)
print()
latitude = data['iss_position']['latitude']
longitude = data['iss_position']['longitude']
zoom = '4'
googleMapUrl = 'http://www.google.com/maps/place/'+latitude+','+longitude+'/@'+latitude+','+longitude+','+zoom+'z'
print(googleMapUrl)  # not necessary to print this, but useful for debugging
success = webbrowser.open_new_tab(googleMapUrl)

Generating "pretty" JSON

In [ ]:
import requests
import json

url = 'http://api.open-notify.org/iss-now.json'
response = requests.get(url)
data = response.json()
print('Python data structure: ', data)
print()
# json.dumps creates a JSON string from a Python data structure
# the indent argument sets the indentation spacing and sort_keys alphabetizes the keys
prettifiedJson = json.dumps(data, indent=4, sort_keys=True)
print('prettified JSON: ')
print(prettifiedJson)

# Data from a GitHub Gist

Putting data in a GitHub Gist is about the easiest and fastest way to make data available to anyone to use in Python.  All you need to do is give them the gist URL.

JSON data about Vanderbilt Schools and Colleges:

https://gist.github.com/baskaufs/22555ec59675f8790bf6cdd9cb0f2961

In [ ]:
import requests
import webbrowser

url = 'https://gist.githubusercontent.com/baskaufs/22555ec59675f8790bf6cdd9cb0f2961/raw/6841e723f0356df112a1f75b8993a18bc180c5f4/vandy_schools.json'
response = requests.get(url)
print('Response media type: ', response.headers['content-type'])
data = response.json()
# the JSON is an array of objects, which gets translated into a list of dictionaries Python structure
# each school is one item in the list

schoolName = input('What school do you want to know about? ')
for school in data:
    if schoolName.lower() in school['label'].lower():
        wikidataUrl = 'https://www.wikidata.org/wiki/' + school['wikidataId']
        print(school['label'])
        print(wikidataUrl)
        success = webbrowser.open_new_tab(wikidataUrl)